# Classification with SnapBoost and XGBoost

In this notebook, we will illustrate machine learning classification using two gradient boosting frameworks; XGBoost and SnapBoost. Many thanks to Tom Parnell and IBM Research Zurich who wrote the original notebook from which I adapted this workshop-friendly copy. 
We will compare the generalization capability of both frameworks. More on generalization later...
Gradient boosting machines (GBMs) can be particularly effective when working with tabular data, especially for large and complex datasets. This notebook embraces choice; you can choose one of 9 binary classification datasets from [OpenML](https://www.openml.org/) to explore and then train your models with.

## What is gradient boosting?

Lets start with 'boosting'. Boosting is a method for creating an ensemble of weak simple prediction models (typically decision trees) that when added together create a strong model that fits more accurately to our dataset. 

Now what about gradient boosting?

Gradient boosting involves creating our ensemble of weaker models in a sequential way where each weak models are added to our stronger composite model one after the other to nudge our composite model into being better. This approach of sequentially adding models is benificial because each subsequent model can learn from the mistakes of the previous model. 

As an example, we start by fitting an initial model (e.g. a tree or linear regression) to our data. Then a second model is built that focuses on accurately predicting the cases where the first model performs poorly (observations with the highest error). The combination of these two models is expected to be better than either model alone. Then you repeat this process of boosting many times.  Each successive model attempts to correct for the shortcomings of the combined boosted ensemble of all previous models. 

One caveat here is we need to be careful about overfitting so we need to be clear on our stopping criteria to stop boosting! You also need to tune the GBM to get a good model unlike Random Forest. To tune the hyper-parameters some understanding of underlying maths is required which is beyond the scope of today, however for some fantastic explanations to help build intuition around gradient boosting, I would recommend this article: https://explained.ai/gradient-boosting/index.html

For this demo we'll do the following:

* Choose a dataset
* Explore our data
* Prepare the data
* Define inner cross validation for hyper-parameter tuning 
* Define outer cross validation to estimate generalisation 
* Train our model
* Analyse model performance 

## Installation of required dependencies

Let's begin by importing all necessary modules and functions. We also print out the version of XGBoost and pai4sk we are using to ensure reproducibility.

In [14]:
import sys
!{sys.executable} -m pip install openml

import openml
import pandas as pd
import numpy as np
import xgboost as xgb
import pai4sk
import psutil
import os
import json
import time
import matplotlib.pyplot as plt
from multiprocessing import Process, Queue
from pai4sk.metrics import log_loss
from pai4sk.preprocessing import LabelEncoder
from pai4sk.model_selection import StratifiedKFold
print("Using XGBoost version: %s" % (xgb.__version__))
print("Using pai4sk version:  %s" % (pai4sk.__version__))

Using XGBoost version: 0.90
Using pai4sk version:  1.6.0


## Choose Dataset

There are a number of datasets from OpenML we can use. 
Select the dataset from the corresponding number to use it:

4154 - Credit Card Subset  

1471 - EEG measurement with the Emotiv EEG Neuroheadset 

4534 - Phishing Websites

734 - control data from F16 aircraft

1046 - Mozilla data

1019 - pen digits

959 - nursery admissions

977 - Letter Image Recognition Data

846 - Aircraft control

In [15]:
# Change dataset_id to the above id of the dataset you want to explore.
dataset_id = 734
dataset = openml.datasets.get_dataset(dataset_id)

X, y, categorical_indicator, ft_names = dataset.get_data(target=dataset.default_target_attribute)
A = dataset.get_data(target=dataset.default_target_attribute)

## Explore dataset

### Positive and Negative Class Distribution

We know our labels can either be P or N so lets explore the relative frequencies.

In [16]:
y.value_counts()

N    7922
P    5828
Name: binaryClass, dtype: int64

### Numerical data class skew

If our dataset has numeric columns, we can average those values across both our label categories and check for any class skew.

In [17]:
def skewCheck (X,y):
    df = X._get_numeric_data()
    if df.empty:
        print("Dataframe contains no numeric data.")
    else: 
        df['y'] = y
        return df.groupby('y').mean()
    
skewCheck(X,y)

,climbRate,Sgz,p,q,curPitch,curRoll,absRoll,diffClb,diffRollRate,diffDiffClb,...,diffSeTime7,diffSeTime8,diffSeTime9,diffSeTime10,diffSeTime11,diffSeTime12,diffSeTime13,diffSeTime14,alpha,Se
y,,,,,,,,,,,,,,,,,,,,,
P,8.929822,-14.322752,0.103756,0.107519,0.703116,0.157756,-14.092485,-1.660604,-0.001451,-0.123610,...,-0.000071,0.0,-0.000088,-2.059025e-06,-0.000084,-1.715854e-06,-0.000081,-1.372684e-06,0.719303,0.025514
N,-25.839308,-11.331861,-0.066026,0.027112,0.575184,-0.008748,-8.794749,-0.394724,-0.000587,0.000151,...,-0.000101,0.0,-0.000112,6.311537e-07,-0.000114,-3.786922e-07,-0.000109,-1.262307e-07,0.553105,0.019424


## Prepare dataset

Since ML algorithms works with numbers, we would like to map string or categorical inputs to integers. Any categorical features are encoded using a label encoder, and all labels are encoded so that they either take value 0 or 1:

In [18]:
# Encode our data and then save as a numpy array
for (ft_ind, ft_name) in enumerate(ft_names):
        if categorical_indicator[ft_ind]:
            X[ft_name] = LabelEncoder().fit_transform(X[ft_name])

X = X.values.astype(np.float64)

# encode labels to 0,1
y = np.array(y)
labels = sorted(list(set(y)))
y[y == labels[0]] = 0
y[y == labels[1]] = 1
y = y.astype(np.float64)

# Model Training <a name="model_training"></a>

We have transformed our data and are now ready to build our models, XGBoost and SnapBoost:


### What is XGBoost?

XGBoost is extreme gradient boosting algorithm based on trees and tends to perform very good out of the box compare to other ML algorithms.
XGBoost is popular amongt data-scientist and one of the most common ML algorithms used in [Kaggle](https://www.kaggle.com/) Competitions.

## What is SnapBoost?

SnapBoost is a new boosting algorithm developed by IBM Research in Zurich. It is different to popular boosting algorithms such as XGBoost, CATboost in that rather than our ensemble being made up of models with the same architecture (ie all decision trees with the same depth), SnapBoost builds an ensemble of different architectures (trees of different depth, regression algorithms) to approximate a wider class of functions. Therefore SnapBoost inolves heterogenous gradient boosting, rather than homogeneous. There are some other unique features like early stopping if our model ceases to improve etc. 


## Scoring Functions

We will now define a function that trains an XGBoost model, for a given set of hyper-parameters, on the training set and then evaluates the trained model on the test set. The evaluation metric used in this demo is the logistic loss.

In [19]:
def score_xgb(X_train, y_train, X_test, y_test, params):
    """ Train and evaluate an XGBoost model for a given set of parameters
    
    Args:
        X_train (np.ndarray): Training feature matrix
        y_train (np.ndarray): Training labels
        X_test (np.ndarray): Test feature matrix
        y_test (np.ndarray): Test labels
        params (dict): Hyper-parameters of XGBoost

    Returns:
        score (float): Logistic loss of the trained model evaluated on the test set
        
    """
    dtrain = xgb.DMatrix(X_train, y_train)
    dtest = xgb.DMatrix(X_test, y_test)
    xgb_params = params.copy()
    num_round = xgb_params['num_round']
    xgb_params.pop('num_round')
    bst = xgb.train(xgb_params, dtrain, num_boost_round=num_round)
    z_test = bst.predict(dtest)
    score = log_loss(y_test, z_test.astype(np.float64))
    return score

Next, we define a function that does the same, but for SnapBoost:

In [20]:
def score_snap(X_train, y_train, X_test, y_test, params):
    """ Train and evaluate a SnapBoost model for a given set of parameters
    
    Args:
        X_train (np.ndarray): Training feature matrix
        y_train (np.ndarray): Training labels
        X_test (np.ndarray): Test feature matrix
        y_test (np.ndarray): Test labels
        params (dict): Hyper-parameters of SnapBoost

    Returns:
        score (float): Logistic loss of the trained model evaluated on the test set
        
    """
    bst = pai4sk.BoostingMachine(**params)
    bst.fit(X_train, y_train)
    z_test = bst.predict_proba(X_test)[:,1]
    score = log_loss(y_test, z_test.astype(np.float64))
    return score

We haven't yet talked about splitting our data into test and train...

Since both models have a lot of different hyper-parameters, it is very important that cross-validation (CV) is performed to tune the models to achieve their maximum potential. 

Hold the phone, what is cross validation? 

Well, there are shortcomings to splitting our data into fixed train, test and validation sets and using the same validation set to evaluate and holding back test till train is complete. The performance of our model will be dependent on the specific subsets of data we choose for training and validation. It could be that if we chose a different subset of the data for training and validation that model performance could be quite different. In this case it would be even better if we could split our training and validation sets multiple times with each time being on a different subset of the data, evaluate our model each time and look at the average performance of the models trained over these multiple evaluations. Enter k-fold cross validation ...  

So how does k-fold cross validation work in practice? 

1. Split your data into K ‘folds’ (sections).
2. Train your model using K-1 folds using the parameter value.
3. Test your model on the remaining fold.
4. Repeat steps 3 and 4 so that every fold is the test data once.

We will run the above steps for different random configurations of parameters to find which parameters minimize our loss. 

However, we face one further challenge. Since the datasets have 10k-20k examples, it is important that care is taken when estimating the generalization error. If one only uses a single train/test split, it is possible to obtain results that are highly biased to this particular split. To solve this problem, it is common to use an additional outer CV for measuring the generalization error.

So to summarise, we will perform **nested cross validations**. We split our data into outer folds and then perform inner CV for each outer fold.

## Inner Cross-Validation (Hyper-parameter Tuning)

The inner cross-validation loop is responsible for tuning the hyper-parameters of a given model. It is implemented in a parallel, asynchronous manner. As we will see later, we will spawn many processes, each one responsible for evaluating many different hyper-parameter configurations across different folds. The following functions defines the work performed by each of these worker processes:

In [21]:
def eval(X, y, model, q_in, q_out, cpu=None):
    """ Worker function for evaluating models on cross-validation folds.
    
    This function should be called by a worker process. The function pulls hyper-parameter
    configurations from an input queue, as well as the indices of the training and validation
    set. It then trains a model using the hyper-parameter configuration and scores it on the
    validation set. The result is then pushed into the output queue. This repeats indefinitely
    and the worker process must be terminated from the master.
    
    The process can be optionally bound to a specific cpu core.
    
    Args:
        X (np.ndarray): Training dataset
        y (np.ndarray): Training labels
        model (str): Model selection (XGBoost or SnapBoost)
        q_in (Queue): Input queue
        q_out (Queue): Output queue
        cpu (int): CPU on which to bind
        
    """
    if cpu is not None:
        p = psutil.Process()
        p.cpu_affinity([cpu])
        os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
        os.environ['OMP_NUM_THREADS'] = '1'

    while True:
        msg = q_in.get()
        X_train = X[msg['train_ind'],:]
        y_train = y[msg['train_ind']]
        X_test = X[msg['test_ind'], :]
        y_test = y[msg['test_ind']]
        if model == 'xgb':
            msg['score'] = score_xgb(X_train, y_train, X_test, y_test, msg['params'])
        else:
            msg['score'] = score_snap(X_train, y_train, X_test, y_test, msg['params'])    
        q_out.put(msg)

The next function performs the inner cross-validation for hyper-parameter tuning. It evaluates (in parallel) a given number of random configurations across a number of stratified folds of the training set and identifies the one that gives the minimal validation loss.

In [22]:
def hpt_parallel(X, y, random_state=42, num_procs = 1, 
                 num_configs = 100, model='xgb', num_folds=3):
    
    """ Parallel, asynchronous cross-validation via random search.
    
    This function performs stratified cross-validation using a many processes.
    Each process is locked to a single CPU core.
    
    Args:
        X (np.ndarray): Training dataset
        y (np.ndarray): Training labels
        random_state (int): Random state for reproducibility
        num_procs (int): Number of proceses to use
        num_configs (int): Number of random configurations to draw
        model (str): Model selection (XGBoost or SnapBoost)
        num_folds (int): Number of cross-validation folds
    
    Returns:
        best_param (dict): Best configuration identified
        best_score (float): Validation score for best configuration
    """
    
    # create flat list of all possible depth configurations for SnapBoost
    depth_limit = 20
    depth_configs = []
    for i in range(1, depth_limit):
        for j in range(i, depth_limit):
            depth_configs.append((i,j))

    # splitter for creating folds
    splitter = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=random_state)

    # what is this used for? 
    # fix random state
    np.random.seed(random_state)

    # draw configurations at random
    param_list = []
    for i in range(0, num_configs):        
        num_round = np.random.randint(10, 1000)
        max_depth = np.random.randint(1, depth_limit)
        learning_rate = 10 ** np.random.uniform(-2.5, -1)
        colsample = np.random.uniform(0.5, 1.0)
        subsample = np.random.uniform(0.5, 1.0)
        lambda_l2 = 10 ** np.random.uniform(-2, 2)

        if model == 'xgb':
            params = {
                'objective': 'binary:logistic',
                'num_round': num_round,
                'max_depth': max_depth,
                'tree_method': 'hist',
                'max_bin': 256,
                'nthread': 1,
                'learning_rate': learning_rate,
                'colsample_bytree': colsample,
                'subsample': subsample,
                'lambda': lambda_l2,
                'seed': int(np.random.randint(np.iinfo(np.int32).max))
            }
        else:
        
            config = depth_configs[np.random.randint(len(depth_configs))]

            params = {
                'objective': 'logloss',
                'num_round': num_round,
                'random_state': int(np.random.randint(np.iinfo(np.int32).max)),
                'learning_rate': learning_rate,
                'colsample_bytree': colsample,
                'subsample': subsample,
                'lambda_l2': lambda_l2,
                'hist_nbins': 256,
                'min_max_depth': config[0],
                'max_max_depth': config[1]
            }

        param_list.append(params)

    # create input and output queue
    q_in = Queue()
    q_out = Queue()

    # populate queue for evaluation
    for i, (train_ind, test_ind) in enumerate(splitter.split(X, y)):
        for j, params in enumerate(param_list):
            msg = {
                'param_id': j,
                'fold_id': i,
                'train_ind': train_ind,
                'test_ind': test_ind,
                'params': params
            }
            q_in.put(msg)

    # create worker processes
    procs = []
    for p in range(0, num_procs):
        procs.append(Process(target=eval, args=(X, y, model, q_in, q_out, p)))

    # start workers
    for proc in procs:
        proc.start()

    # collect results from workers
    scores = np.zeros(shape=(num_configs, num_folds))
    for i in range(0, num_configs*num_folds):
        print('Iteration', i, end='\r')
        msg = q_out.get()
        param_id = msg['param_id']
        fold_id = msg['fold_id']
        scores[param_id, fold_id] = msg['score']

    # average over folds
    scores = np.mean(scores, axis=1)

    # identify best configuration
    best_param_id = np.argmin(scores)

    # terminate worker processes
    for proc in procs:
        proc.terminate()

    return param_list[best_param_id], scores[best_param_id]

## Outer Cross-Validation (Generalization Estimation)

In order to estimate the generalization performance of a tuned model, we also need to use an additional layer of stratified cross-validation. For each outer fold, we perform the inner CV on the training set to identify the optimal parameters. After we have identified the optimal configuration, we need to re-train that configuration on the entirety of the training set, and evaluate it on the corresponding test set. This process is repeated for all outer folds - creating k models, k performances with the generalization performance given as the average across all outer folds.

The following function performs the task of re-training the best model found by the inner CV:

In [23]:
def eval_params(X_train, y_train, X_test, y_test, params, model):
    """ Evaluate the best configuration on a given outer fold.
    
    This function evaluates a single hyper-parameter configuration of a given model, by 
    training on a training set and scoring on a test set. The training and evaluation 
    is performed within a separate process.
    
    Args:
        X_train (np.ndarray): Training dataset
        y_train (np.ndarray): Training labels
        X_test (np.ndarray): Test dataset
        y_test (np.ndarray): Test labels
        params (dict): Hyper-parameter configuration
        model (str): Model selection (XGBoost or SnapBoost)
    
    Returns:
        score (float): Logistic loss of the trained model evaluated on the test set
    """
    q = Queue()
    def p_func(X_train, y_train, X_test, y_test, params, model, q):
        if model == 'xgb':
            score = score_xgb(X_train, y_train, X_test, y_test, params)
        else:
            score = score_snap(X_train, y_train, X_test, y_test, params)  
        q.put({'score': score})
    p = Process(target=p_func, args=(X_train, y_train, X_test, y_test, params, model, q))
    p.start()
    msg = q.get()
    p.join()
    return msg['score']

We now come to the main part of the demo, which performs nested CV for the list of Open ML datasets provided and compares the loss achieved by both XGBoost and SnapBoost. If you wish to evalaute a particular data only, a subset of the datasets, or a different set of datasets, simply modify `DATASET_IDS` accordingly.

In [24]:
# experiment parameters:
NUM_OUTER_FOLDS=3 # number of outer folds
NUM_INNER_FOLDS=3 # number of inner folds
NUM_CONFIGS=100 # number of configurations to try
# provide number of cores for our machine ... 
NUM_PROCS = 40 # number of processes/cores
PRINT_PARAMS = True # print off best parameters found

# DataFrame for collecting results for all datasets
df = pd.DataFrame()

# counter to keep track of time
t_tot = 0

# DATASET_IDS = [dataset]
name = dataset.name 

# random state 
seed = dataset_id 

In [ ]:
# create statified kfold spliter
splitter = StratifiedKFold(n_splits=NUM_OUTER_FOLDS, shuffle=True, random_state=seed)

# start nested CV
t0 = time.time()
print(">> Starting nested CV on dataset: %s" % (name))

# DataFrame for collecting results for this dataset
this_df = pd.DataFrame()

# Outer Loop: outer cross validation 
for fold, (train_ind, test_ind) in enumerate(splitter.split(X, y)):

    # results for this dataset, containing fold
    res = pd.Series(name=fold, dtype=np.float64)

    # create train/test split for this outer fold
    X_train, X_test = X[train_ind, :], X[test_ind, :]
    y_train, y_test = y[train_ind], y[test_ind]

    # loop over both models
    for model in ['xgb','snap']:

    # Inner Loop: perform inner CV for this outer fold
        opt_params, res["val_%s" % (model)] = hpt_parallel(X_train, y_train, 
                                                           random_state=seed,
                                                           num_procs=NUM_PROCS, 
                                                           num_configs=NUM_CONFIGS, 
                                                           model=model, 
                                                           num_folds=NUM_INNER_FOLDS)

        # optionally print off the best configuration found
        if PRINT_PARAMS:
            print("Best parameters for model %s on outer fold %d:" % (model, fold))
            print(json.dumps(opt_params, indent=4))

        # re-train the best configuration on the train set and evaluate on the test set
        res["test_%s" % (model)] = eval_params(X_train, y_train, 
                                               X_test, y_test, 
                                               params=opt_params, 
                                               model=model)
        
        # store results for this fold
    this_df = this_df.append(res)
    
t_elap = time.time() - t0
t_tot += t_elap
print(">> Finished nested CV on dataset %s in %d seconds" % (name, t_elap))

>> Starting nested CV on dataset: ailerons
Best parameters for model xgb on outer fold 0:
{
    "objective": "binary:logistic",
    "num_round": 531,
    "max_depth": 5,
    "tree_method": "hist",
    "max_bin": 256,
    "nthread": 1,
    "learning_rate": 0.030576966439305724,
    "colsample_bytree": 0.7157460227581843,
    "subsample": 0.9569441240285168,
    "lambda": 2.214122307473504,
    "seed": 694466545
}
Best parameters for model snap on outer fold 0:
{
    "objective": "logloss",
    "num_round": 495,
    "random_state": 1640255808,
    "learning_rate": 0.023748091051762912,
    "colsample_bytree": 0.7527176254754524,
    "subsample": 0.5901700545531336,
    "lambda_l2": 1.241457692562928,
    "hist_nbins": 256,
    "min_max_depth": 1,
    "max_max_depth": 6
}


In [ ]:
this_res = this_df.mean()
this_res['name'] = dataset.name
df = this_res

In [ ]:
# val_loss_gain = 100*((df['val_xgb']-df['val_snap'])/df['val_xgb'])
# test_loss_gain = 100*((df['test_xgb']-df['test_snap'])/df['test_xgb'])

max_val = max(df['val_xgb'], df['val_snap'])
min_val = min(df['val_xgb'], df['val_snap'])

max_test = max(df['test_xgb'], df['test_snap'])
min_test = min(df['test_xgb'], df['test_snap'])

val_loss_gain = 100*((max_val-min_val)/max_val)
test_loss_gain = 100*((max_test-min_test)/max_test)

# for whichever value is larger 

if (df['val_xgb'] > df['val_snap']): 
    val_outperformer = "SnapBoost"
else: 
    val_outperformer = "XGBoost"

if (df['test_xgb'] > df['test_snap']): 
    test_outperformer = "SnapBoost"
else: 
    test_outperformer = "XGBoost"
    
print("For the " + dataset.name + " dataset we see " + val_outperformer + " outperforms on the validation set providing a " + str(val_loss_gain) + "% improvement in validation loss.")
print("For the " + dataset.name + " dataset we see " + test_outperformer + " outperforms on the test set providing a " + str(test_loss_gain) + "% improvement in generalization loss.")


